# Exercise 04 - Due Friday, October 21 at 12pm

*Objectives*: Gain experience loading a CSV dataset into a database model you define yourself and using SQL to explore its contents. Explore the data by writing and executing a number of SQL queries using common syntax and functions and describing your findings.

*Grading criteria*: The tasks should all be completed, and questions should all be answered with SQL queries in the space provided, unless a text answer is requested. Results should be correct, according to the embedded tests. The notebook itself should be completely reproducible; from start to finish, another person should be able to use the same code to obtain the same results as yours.  Note that you will receive no more than partial credit if you do not add text/markdown cells explaining your thinking at each major step in each problem.

*Deadline*: Friday, October 21, 12pm. Submit your notebook to Blackboard and push it to your GitHub repository.

## Setup - obtain data and create database

The US City Open Data Census has data on [restaurant inspections from all over the country](http://us-city.census.okfn.org/dataset/food-safety).  Let's take a look at data from Albuquerque.  *Note*: metadata and other details for this dataset are available at http://data.cabq.gov/business/foodinspections/.

Fyi, to save you a step, I've converted this file into a clean CSV file by using the `dos2unix` command to change its line endings and to switch it to use commas instead of tabs using `csvformat -t inputfile.csv > outputfile.csv`.

First we download the dataset:

In [2]:
!echo 'redspot' | sudo -S service postgresql restart

[sudo] password for jovyan: Restarting PostgreSQL 9.5 database server: main.


In [3]:
!createdb -U dbuser food_inspection

In [4]:
!wget https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/exercises/abq-food-inspections-fy2010.csv

--2016-10-24 04:04:15--  https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/exercises/abq-food-inspections-fy2010.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.32.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.32.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5025226 (4.8M) [text/plain]
Saving to: ‘abq-food-inspections-fy2010.csv’

abq-food-inspection 100%[=====================>]   4.79M  --.-KB/s   in 0.08s  

2016-10-24 04:04:15 (60.4 MB/s) - ‘abq-food-inspections-fy2010.csv’ saved [5025226/5025226]



## Problem 1 - examine the data (35 points)

Use `csvstat` and other tools as you see fit and observe its contents.  Review the metadata definitions linked to above.  Describe what you see:  What do all the columns mean?  Are there null values?  Which columns interest you the most?  Which columns present some complications or questions you would like to ask?

In [5]:
!wc -l abq-food-inspections-fy2010.csv

17557 abq-food-inspections-fy2010.csv


Let's rename the csv file for our convenience.

In [6]:
!mv abq-food-inspections-fy2010.csv food.csv

Now, let's see all the column headers in this csv file.

In [6]:
!csvcut -n food.csv

  1: FACILITY_NAME
  2: FACILITY_KEY
  3: SITE_ADDRESS
  4: CITY
  5: STATE
  6: ZIP
  7: OWNER_KEY
  8: OWNER_NAME
  9: NATURE_OF_BUSINESS
 10: STREET_NUMBER
 11: STREET_NAME
 12: STREET_TYPE
 13: POST_DIRECTIONAL
 14: PHONE
 15: PROGRAM_CATEGORY
 16: PROGRAM_CATEGORY_DESCRIPTION
 17: INSPECTION_DATE
 18: INSPECTION_TYPE
 19: INSPECTION_DESC
 20: SERIAL_NUM
 21: ACTION_CODE
 22: ACTION_DESC
 23: RESULT_CODE
 24: RESULT_DESC
 25: VIOLATION_CODE
 26: VIOLATION_DESC
 27: INSPECTION_MEMO


In [7]:
!shuf -n 100 food.csv | csvstat

  1. MARIOS PIZZA AND PASTA EXPRESS INC
	<class 'str'>
	Nulls: False
	Unique values: 96
	5 most frequent values:
		OOB CAROM CLUB LLC:	2
		99 BANH SUPERMARKET:	2
		THE CUBE RESTAURANT:	2
		HASTINGS BOOKS MUSIC & VIDEO:	1
		KASEMAN PRESBYTERIAN HOSPITAL:	1
	Max length: 59
  2. 85411
	<class 'int'>
	Nulls: False
	Min: 429
	Max: 92750
	Sum: 4060474
	Mean: 41014.88888888889
	Median: 40093
	Standard Deviation: 30419.53572152125
	Unique values: 96
	5 most frequent values:
		18468:	2
		72960:	2
		66401:	2
		13595:	1
		24645:	1
  3. 5700 4TH ST NW STE F
	<class 'str'>
	Nulls: False
	Unique values: 95
	5 most frequent values:
		5315 GIBSON BLVD SE:	2
		301 2L CENTRAL ST NW:	2
		1520 CENTRAL AV SE:	2
		6001 LOMAS BLVD NE:	2
		10001 COORS BLVD NW:	1
	Max length: 28
  4. ALBUQUERQUE
	<class 'str'>
	Nulls: False
	Values: ALBUQUERQUE
  5. NM
	<class 'str'>
	Nulls: False
	Values: NM
  6. 87107
	<class 'int'>
	Nulls: False
	Min: 87102
	Max: 87123
	Sum: 8623966
	Mean: 87110.76767676767
	Median: 87110
	

Using csvstat we got data about all 27 coloumns in the csv file. The Above csvstat analysis shows the present of nulls in certain columns. It also tells the type of values in that particular column and the maximum length as well.

I performed csvstat on the three columns of intersest to me. It tells us, which facility was inspected on which date. It also has the inspection type column which is wholly null.

In [8]:
!head -1000 food.csv | csvcut -c1,17,18 | csvstat 

  1. FACILITY_NAME
	<class 'str'>
	Nulls: False
	Unique values: 126
	5 most frequent values:
		99 BANH SUPERMARKET:	57
		ALBERTSONS 904:	28
		ALBERTSONS 922:	23
		ALBERTSONS 937:	22
		ALBERTSONS 911:	22
	Max length: 34
  2. INSPECTION_DATE
	<class 'datetime.date'>
	Nulls: False
	Min: 2009-07-01
	Max: 2010-06-24
	Unique values: 159
	5 most frequent values:
		2009-12-30:	31
		2009-12-23:	29
		2010-01-29:	27
		2009-07-21:	26
		2009-07-29:	20
  3. INSPECTION_TYPE
	<class 'NoneType'>
	Nulls: True
	Values: 

Row count: 999


Csvcut code to look at different columns of interest to us.

In [17]:
!csvcut -c1,17,25 food.csv | head -10 | csvlook

|--------------------------+---------------------+-----------------|
|  FACILITY_NAME           | INSPECTION_DATE     | VIOLATION_CODE  |
|--------------------------+---------------------+-----------------|
|  00BSAY IT WITH CAKE     | 2009-10-21 00:00:00 |     68          |
|  00BSAY IT WITH CAKE     | 2009-10-21 00:00:00 |     00          |
|  00BSAY IT WITH CAKE     | 2009-10-21 00:00:00 |     55          |
|  00BSAY IT WITH CAKE     | 2010-04-14 00:00:00 |     00          |
|  00BSAY IT WITH CAKE     | 2010-04-14 00:00:00 |     55          |
|  2000 VIETNAM RESTAURANT | 2010-03-23 00:00:00 |     34          |
|  2000 VIETNAM RESTAURANT | 2010-03-23 00:00:00 |     35          |
|  2000 VIETNAM RESTAURANT | 2010-03-23 00:00:00 |     55          |
|  328 CHINESE CUISINE     | 2009-12-15 00:00:00 |     68          |
|--------------------------+---------------------+-----------------|


Code to find out, how many times facility 7 eleven was inspected and when.

In [18]:
!csvcut -c1,17,25 food.csv | csvgrep -c1 -m '7 ELEVEN' | csvsort -c2 | head -10 | csvlook


|----------------+-----------------+-----------------|
|  FACILITY_NAME | INSPECTION_DATE | VIOLATION_CODE  |
|----------------+-----------------+-----------------|
|  7 ELEVEN 706  | 2009-07-09      | 00              |
|  7 ELEVEN 706  | 2009-07-09      | 55              |
|  7 ELEVEN 707  | 2009-07-14      | 02              |
|  7 ELEVEN 707  | 2009-07-14      | 55              |
|  7 ELEVEN 719  | 2009-07-23      | 02              |
|  7 ELEVEN 719  | 2009-07-23      | 55              |
|  7 ELEVEN 700  | 2009-07-27      | 02              |
|  7 ELEVEN 700  | 2009-07-27      | 55              |
|  7 ELEVEN 709  | 2009-07-28      | 00              |
|----------------+-----------------+-----------------|


Insert a combination of code and text/markdown cells here to review the data and describe what you find.

## Problem 2 - define a database model and load the data (35 points)

Based on what you found above, create and connect to a new database, define a database table in it, and load this dataset into it.  You may use either of the methods for this step you have seen in earlier class notebooks.  You may choose to eliminate variables/columns if they are not relevant or interesting to you - explain your reasoning if you do.  Either way, you should load a majority of the columns present in the source dataset into the database and all of its rows.

Once your data has loaded successfully, run a `COUNT(*)` query to verify that all the data has loaded correctly.

Insert a combination of code and text/markdown cells here to connect to your database, define your table, load its data, and verify the loaded records.

In [9]:
%load_ext sql

/opt/conda/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/opt/conda/lib/python3.5/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [10]:
%sql postgresql://dbuser@localhost:5432/food_inspection

'Connected: dbuser@food_inspection'

Since we have already created the database and connected it, let's make a table to load all the csv file (i.e. food) data into it.

In [11]:
%%sql
DROP TABLE IF EXISTS food;
CREATE TABLE food(
  FACILITY_NAME VARCHAR(256),
  FACILITY_KEY INTEGER,
  SITE_ADDRESS VARCHAR(128),
  CITY VARCHAR(64),
  STATE VARCHAR(64),
  ZIP VARCHAR(64),
  OWNER_KEY INTEGER,
  OWNER_NAME VARCHAR(256),
  BUSINESS_NATURE VARCHAR(128),
  ST_NUM VARCHAR(64),
  ST_NAME VARCHAR(64),
  ST_TYPE VARCHAR(64),
  POST_DIRECTION VARCHAR(64),
  PHONE VARCHAR(64),
  PROGRAM_CATEGORY_ID INTEGER,
  PROGRAM_CATEGORY_DESC VARCHAR(128),
  INSPECTION_DATE TIMESTAMP,
  INSPECTION_TYPE VARCHAR(16),
  INSPECTION_DESC VARCHAR(128),
  SERIAL_NUM VARCHAR(64),
  ACTION_CODE VARCHAR(64),
  ACTION_DESC VARCHAR(128),
  RESULT_CODE INTEGER,
  RESULT_DESC VARCHAR(128),
  VIOLATION_CODE VARCHAR(64),
  VIOLATION_DESC VARCHAR(128),
  INSPECTION_MEMO VARCHAR(1024)
)

Done.
Done.


[]

In [12]:
!pwd

/home/jovyan/work


Now, we will load the data into the table that we created using sql.

In [13]:
%%sql
COPY food FROM '/home/jovyan/work/food.csv'
CSV
HEADER
QUOTE '"'
DELIMITER ',';

17556 rows affected.


[]

In [14]:
%%sql
SELECT COUNT(*) FROM food;

1 rows affected.


count
17556


The count from the above sql query is same as the number of rows affected while loading the data into the table. This means we have succesfully loaded the csv data into the table in our database.

## Problem 3 - explore your data (30 points)

Now that the data is loaded, ask some questions of it!  Identify key variables of interest and note their ranges along with other useful descriptive statistics.  Choose and define a few lines of inquiry, execute queries for each, and describe what you find in your result sets.  Use any query techniques we've seen in class, including aggregate functions, transformations, subqueries, or others as appropriate. 

If you find interesting patterns, adding plots to your exploration would be useful.

Insert a combination of code and text/markdown cells here to explore your data.

Now, lets explore the data we just loaded in our database. I've set limit to 10 in all the queries just for the convenience.

In [15]:
%%sql
SELECT DISTINCT PROGRAM_CATEGORY_ID, PROGRAM_CATEGORY_DESC
FROM food
ORDER BY PROGRAM_CATEGORY_DESC
LIMIT 10;

10 rows affected.


program_category_id,program_category_desc
403,Bakery
501,Bar
402,Childcare
405,Food Processor
406,Food Service Establishment
1003,Food/Drink Vending - Up to 15 Units
1000,Frozen Food Truck
404,Institutional Kitchen
407,Limited Menu
602,Meat Market


In [16]:
%%sql
SELECT DISTINCT BUSINESS_NATURE
FROM food
LIMIT 10;

10 rows affected.


business_nature
None
DIST. SERVICE
COOKING CONSULTANT
REDI MART
MUR/RESTAURANT
RETAIL COFFEE SALES
RETAIL COFFEE/FOOD
BAR/PROC/REST
WHOLE SALE SEAFOOD
SCHOOL


A general query to list all important columns in the table.

In [17]:
%%sql
SELECT OWNER_NAME, FACILITY_NAME, BUSINESS_NATURE, INSPECTION_DATE, PROGRAM_CATEGORY_DESC, VIOLATION_CODE
FROM food
GROUP BY OWNER_NAME,BUSINESS_NATURE, FACILITY_NAME, INSPECTION_DATE, PROGRAM_CATEGORY_DESC, vIOLATION_CODE
ORDER BY OWNER_NAME
LIMIT 10;

10 rows affected.


owner_name,facility_name,business_nature,inspection_date,program_category_desc,violation_code
110 SUNPORT LLC,HOLIDAY INN EXPRESS,HOTEL,2009-09-11 00:00:00,Food Service Establishment,55
110 SUNPORT LLC,HOLIDAY INN EXPRESS,HOTEL,2010-03-29 00:00:00,Food Service Establishment,55
110 SUNPORT LLC,HOLIDAY INN EXPRESS,HOTEL,2009-08-19 00:00:00,Food Service Establishment,None
110 SUNPORT LLC,HOLIDAY INN EXPRESS,HOTEL,2010-03-29 00:00:00,Food Service Establishment,00
110 SUNPORT LLC,HOLIDAY INN EXPRESS,HOTEL,2009-09-11 00:00:00,Food Service Establishment,00
1ST CLASS LEARNING CENTER,OOB 1ST CLASS LEARNING CENTER,CHILDCARE CENTER & PR,2009-08-18 00:00:00,Childcare,00
1ST CLASS LEARNING CENTER,OOB 1ST CLASS LEARNING CENTER,CHILDCARE CENTER & PR,2010-01-21 00:00:00,Childcare,55
1ST CLASS LEARNING CENTER,OOB 1ST CLASS LEARNING CENTER,CHILDCARE CENTER & PR,2010-01-21 00:00:00,Childcare,40
1ST CLASS LEARNING CENTER,OOB 1ST CLASS LEARNING CENTER,CHILDCARE CENTER & PR,2010-01-21 00:00:00,Childcare,38
1ST CLASS LEARNING CENTER,OOB 1ST CLASS LEARNING CENTER,CHILDCARE CENTER & PR,2009-08-18 00:00:00,Childcare,55


Query to check which facility name has occurred how many times. The result of this query shows the top 10 most frequently inspected facilities.

In [18]:
%%sql
SELECT FACILITY_NAME, COUNT(*)AS FACILITY_COUNT, ST_NAME, ST_NUM
FROM food
GROUP BY FACILITY_NAME, ST_NAME, ST_NUM
ORDER BY FACILITY_COUNT DESC
LIMIT 10;

10 rows affected.


facility_name,facility_count,st_name,st_num
ISOTOPES PARK - OVATIONS,93,AVENIDA CESAR CHAVEZ,1601
PER EHSII CECELIA GARCIA OOB LA REYNA DEL SUR,85,OLD COORS,644
ISLETA AMPHITHEATER,68,UNIVERSITY,5601
ECHO INC,68,MENAUL,300
CIRCLE K 8936,66,CENTRAL,5210
OOB CALIFORNIA DEL NORTE ICE CREAM LLC,65,4TH,1415
TALIN MARKET WORLD FOOD FARE,64,LOUISIANA,88
CIRCLE K 8940,61,CENTRAL,8601
OOB MOM THAI FOOD,61,SAN PEDRO,1212
DIONS PIZZA,61,4TH,6308


A query to check where the violation code is null.

In [19]:
%%sql
SELECT FACILITY_NAME, VIOLATION_CODE
FROM food
WHERE VIOLATION_CODE IS NULL
ORDER BY FACILITY_NAME
LIMIT 10;

10 rows affected.


facility_name,violation_code
5 STAR BURGERS,None
ABC PRESCHOOL,None
ABF FUELING LLC,None
ABF FUELING LLC,None
ACAPULCO MEXICAN RESTAURANT,None
AFC SUSHI AT SMITHS 427,None
ALBERTSONS 915,None
ALBERTSONS 922,None
ALBERTSONS 924,None
ALBERTSONS 937,None


WILDCARD query to see certain type of violation description.

In [26]:
%%sql
SELECT FACILITY_NAME, VIOLATION_DESC
FROM food
WHERE VIOLATION_DESC LIKE '%hand%'
LIMIT 10;


10 rows affected.


facility_name,violation_desc
7 ELEVEN 705,Improper storage/handling of clean equipment/utens
7 ELEVEN 710,Improper storage/handling of clean equipment/utens
7 ELEVEN 711,Improper storage/handling of clean equipment/utens
99 BANH SUPERMARKET,Improper storage/handling of clean equipment/utens
99 BANH SUPERMARKET,Improper storage/handling of clean equipment/utens
99 BANH SUPERMARKET,Improper storage/handling of clean equipment/utens
ALBERTSONS 928,Improper storage/handling of clean equipment/utens
ALBUQUERQUE INDIAN CENTER,Improper storage/handling of clean equipment/utens
ALLSUPS 218,Improper handwashing
ANODYNE,Barehand Contact with ready to eat food


## Bonus (10 points)

We have seen a few simple techniques for normalizing data in class.  Choose a column or set of columns that might be redundant and normalize them out to another table:

* Identify the columns to normalize in the source data table
* Create one or more new tables to hold the normalized records
* Extract data into the new tables
* Re-insert the new foreign key references from the new table into the source table